In [1]:
import time
import numpy as np
import pandas as pd

from tqdm import tqdm


# 데이터 로드

In [2]:
def read_data(route):
    data = pd.read_csv(route)
    data = data[data['ts'] != 'ts']
    data['ts'] = data['ts'].astype('int64')
    # data['time_index'] = ((data['ts'] - data['ts'][0]) / 300) * 5
    
    return data

In [3]:
# 두 데이터 프레임을 ts 기준으로 병합 (하루전가격 기준으로 병합)
기상실측데이터1 = read_data('../OIBC_2024_DATA/data/기상실측데이터_1.csv')
기상실측데이터2 = read_data('../OIBC_2024_DATA/data/기상실측데이터_2.csv')
기상실측데이터 = pd.concat([기상실측데이터1, 기상실측데이터2], ignore_index=True)

# location에 대한 one-hot encoding
# ['Ilgwa-ri' 'Geumak-ri' 'Yongsu-ri' 'Hacheon-ri'
# 'Cheonji-dong' 'Songdang-ri' 'Bonggae-dong' 'Gwangryeong-ri' 'Sangmo-ri']
locations = 기상실측데이터['location'].unique()
기상실측데이터 = pd.get_dummies(기상실측데이터, columns=['location'])
기상실측데이터 = 기상실측데이터.fillna(0)

기상실측데이터

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Cheonji-dong,location_Gaigeturi,location_Geumak-ri,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri
0,1709222220,6.9,2.4,2.4,69.0,1.5,338.0,18.7,37.7,0.0,...,False,False,False,False,False,True,False,False,False,False
1,1709222580,7.1,2.7,2.7,68.0,1.5,338.0,18.7,37.7,0.0,...,False,False,False,False,False,True,False,False,False,False
2,1709226120,6.3,0.9,0.9,64.0,-0.1,338.0,22.0,22.0,0.0,...,False,False,False,False,False,True,False,False,False,False
3,1709229420,5.6,-1.7,-1.7,63.0,-0.9,338.0,31.7,31.7,0.0,...,False,False,False,False,False,True,False,False,False,False
4,1709229840,5.6,-1.7,-1.7,63.0,-0.9,338.0,31.7,31.7,0.0,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98663,1729600283,21.96,0,0,0,0,270.0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
98664,1729602067,19.96,0,0,0,0,300.0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
98665,1729603843,19.96,0,0,0,0,290.0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
98666,1729605674,20.29,0,0,0,0,299.0,0,0,0,...,False,False,False,False,False,False,True,False,False,False


In [4]:
하루전가격 = pd.read_csv('../OIBC_2024_DATA/data/제주전력시장_시장전기가격_하루전가격.csv')
# 하루전가격 = 하루전가격.drop(0)  # 첫 데이터는 학습 데이터가 없어서 제거
하루전가격

,ts,하루전가격(원/kWh)
0,1709218800,107.39
1,1709222400,107.39
2,1709226000,95.30
3,1709229600,87.89
4,1709233200,0.00
...,...,...
5660,1729594800,156.68
5661,1729598400,152.54
5662,1729602000,106.92
5663,1729605600,150.97


# 데이터 병합

In [9]:
data = pd.merge(기상실측데이터, 하루전가격, on="ts", how="outer")

# location에 따른 interpolation
# for location in locatio합location_{location}'] == True]
#     rows = rows.interpolate(method='linear')
#     # print(type(rows), rows[0])
#     data = pd.concat([data, rows], ignore_index=True)
    
# nan 가진 데이터 제거 
# data = data.dropna(axis=0)

# ts를 기준으로 sort
data = data.sort_values('ts')
data.reset_index(inplace=True)
data = data.drop(columns=['index'])

data

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Gaigeturi,location_Geumak-ri,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri,하루전가격(원/kWh)
0,1709218800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.39
1,1709218850,3.23,0,0,0,0,343.0,0,0,0,...,False,False,False,False,False,False,False,False,False,NaN
2,1709218851,7.18,0,0,0,0,341.0,0,0,0,...,False,False,False,False,False,True,False,False,False,NaN
3,1709218856,8.03,0,0,0,0,352.0,0,0,0,...,True,False,False,False,False,False,False,False,False,NaN
4,1709218980,8.5,8.0,8.0,81.0,5.5,270.0,6.1,17.6,0.0,...,False,False,False,False,False,False,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104270,1729607448,15.96,0,0,0,0,290.0,0,0,0,...,False,False,False,False,False,False,False,False,False,NaN
104271,1729607451,20.03,0,0,0,0,290.0,0,0,0,...,True,False,False,False,False,False,False,False,False,NaN
104272,1729608960,20.0,16.6,16.6,82.0,17.2,293.0,33.3,55.6,0.0,...,False,False,False,False,False,False,False,False,False,NaN
104273,1729608960,18.8,14.9,14.9,100.0,18.8,293.0,69.2,69.2,0.0,...,False,False,False,False,False,False,False,False,True,NaN


# Interpolation Class

In [107]:
def sort_values_and_remove_index(data):
    new_data = data.copy()
    new_data = new_data.sort_values('ts')
    new_data.reset_index(inplace=True)
    new_data.drop(columns=['index'], inplace=True)
    
    return new_data


class InterpolationHelper:
    def __init__(self, data, column_names, data_filter=None):
        self.data = data
        self.column_names = column_names
        self.value_finder = data_filter
        
        # 만약 특정 Location 같은 곳 안에서만 필터링 하고 싶으면 적용함
        if self.value_finder is not None:
            self.data = self.value_finder(data).copy()
            self.data.reset_index(inplace=True)
            self.data.drop(columns=['index'], inplace=True)
            
        for column_name in self.column_names:
            self.data[column_name] = self.data[column_name].astype('float64')
            
    def calculate_interpolation_by_value(self, index1, index2, index3, column_name):
        # 결측치 또는 불완전한 값은 필터링
        if index1 is None and index3 is None:
            return None
        if index1 is None:
            return self.data.iloc[index3, :][column_name]
        if index3 is None:
            return self.data.iloc[index1, :][column_name]
        
        time1 = self.data.iloc[index1, :]['ts']
        time2 = self.data.iloc[index2, :]['ts']
        time3 = self.data.iloc[index3, :]['ts']
        
        column1 = self.data.iloc[index1, :][column_name]
        column2 = self.data.iloc[index2, :][column_name]
        column3 = self.data.iloc[index3, :][column_name]
        
        # print(column1, column2, column3, time1, time2, time3)
        
        # nan 아니면 기존 값 사용
        if pd.notna(column2):
            return column2
    
        if time3 > time1:
            diff = (time2 - time1) / (time3 - time1)
            return column1 + diff * (column3 - column1)
        else:
            return column1
        
    def find_value_notna(self, index, column_name):
        # 앞쪽에서 가장 가까운 NaN이 아닌 값의 인덱스 찾기
        previous_non_nan_index = self.data[column_name][:index].last_valid_index()
    
        # 뒤쪽에서 가장 가까운 NaN이 아닌 값의 인덱스 찾기
        if index + 1 >= len(self.data):
            return previous_non_nan_index, None
        
        next_non_nan_index = self.data[column_name][index + 1:].first_valid_index()
        
        return previous_non_nan_index, next_non_nan_index
        
    def calculate_interpolation(self):
        new_data = self.data.copy()
        new_data = sort_values_and_remove_index(new_data)
        
        for index, row in tqdm(new_data.iterrows(), ncols=75, total=len(new_data)):
            for column_name in self.column_names:
                if pd.notna(row[column_name]):
                    continue
                previous_index, next_index = self.find_value_notna(index, column_name)
                next_value = self.calculate_interpolation_by_value(previous_index, index, next_index, column_name)
                new_data.loc[index, column_name] = next_value
            
            
        return new_data

# interpolation_helper_krw = InterpolationHelper(data, '하루전가격(원/kWh)', lambda x: x['location_Bonggae-dong'])

# 각 location에 대한 새 데이터 생성

In [105]:
interpolation_data = data.copy()

price_data = interpolation_data[pd.notna(interpolation_data['하루전가격(원/kWh)'])]

# 한 가격에 대해 모든 지역에서의 데이터를 각각 생성합니다
for location in locations:
    data_inner = price_data.copy()
    for inner_location in locations:
        data_inner[f'location_{inner_location}'] = False
    
    data_inner[f'location_{location}'] = True
    interpolation_data = pd.concat([interpolation_data, data_inner], ignore_index=True)
    
interpolation_data.dropna(subset=[f'location_{locations[0]}'], inplace=True)
interpolation_data = sort_values_and_remove_index(interpolation_data)
interpolation_data

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Gaigeturi,location_Geumak-ri,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri,하루전가격(원/kWh)
0,1709218800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,107.39
1,1709218800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,True,False,False,107.39
2,1709218800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,True,False,False,False,107.39
3,1709218800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,107.39
4,1709218800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,107.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171179,1729609200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,True,False,False,False,False,121.78
171180,1729609200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,False,False,False,False,False,False,False,121.78
171181,1729609200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,True,False,False,False,False,False,False,121.78
171182,1729609200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,True,False,False,False,False,False,121.78


# 데이터 Interpolation (가격제외)

In [112]:
columns = [
    'temp', 'real_feel_temp','real_feel_temp_shade','rel_hum','dew_point',
    'wind_dir','wind_spd','wind_gust_spd','uv_idx','vis','cld_cvr','ceiling',
    'pressure','appr_temp','wind_chill_temp','wet_bulb_temp','precip_1h',
    'cloud','temp_max','temp_min','humidity','ground_press','wind_speed',
    'rain','snow'
]
print('columns:', len(columns), columns)
print('locations:', len(locations), locations)

new_interpolation_data = None
for location in locations:
    interpolation_helper_loc = InterpolationHelper(
        interpolation_data, 
        columns, 
        lambda x: x[x[f'location_{location}'] == True]
    )
    new_data = interpolation_helper_loc.calculate_interpolation()
    
    if new_interpolation_data is None:
        new_interpolation_data = new_data
    else:
        new_interpolation_data = pd.concat([new_interpolation_data, new_data], ignore_index=True)
            
    # new_interpolation_data.to_csv('test.csv', index=False)
            
new_interpolation_data = sort_values_and_remove_index(new_interpolation_data)
new_interpolation_data

columns: 25 ['temp', 'real_feel_temp', 'real_feel_temp_shade', 'rel_hum', 'dew_point', 'wind_dir', 'wind_spd', 'wind_gust_spd', 'uv_idx', 'vis', 'cld_cvr', 'ceiling', 'pressure', 'appr_temp', 'wind_chill_temp', 'wet_bulb_temp', 'precip_1h', 'cloud', 'temp_max', 'temp_min', 'humidity', 'ground_press', 'wind_speed', 'rain', 'snow']
locations: 12 ['Ilgwa-ri' 'Geumak-ri' 'Yongsu-ri' 'Hacheon-ri' 'Cheonji-dong'
 'Songdang-ri' 'Bonggae-dong' 'Gwangryeong-ri' 'Sangmo-ri' 'Gaigeturi'
 'Cheju-do' 'Jeju']


100%|███████████████████████████████| 17382/17382 [00:55<00:00, 310.99it/s]


,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Gaigeturi,location_Geumak-ri,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri,하루전가격(원/kWh)
0,1709218800,6.90,2.4,2.4,69.0,1.5,338.0,18.7,37.7,0.0,...,False,False,False,False,True,False,False,False,False,107.39
1,1709218800,7.18,0.0,0.0,0.0,0.0,341.0,0.0,0.0,0.0,...,False,False,False,False,False,True,False,False,False,107.39
2,1709218800,7.70,0.0,0.0,67.0,1.9,270.0,40.8,43.2,0.0,...,False,False,False,False,False,False,False,False,True,107.39
3,1709218800,3.23,0.0,0.0,0.0,0.0,343.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,107.39
4,1709218800,6.10,1.1,1.1,80.0,2.9,158.0,18.0,18.0,0.0,...,False,False,False,True,False,False,False,False,False,107.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171179,1729609200,20.03,0.0,0.0,0.0,0.0,290.0,0.0,0.0,0.0,...,True,False,False,False,False,False,False,False,False,121.78
171180,1729609200,17.80,13.5,13.5,87.0,15.7,203.0,43.7,66.0,0.0,...,False,True,False,False,False,False,False,False,False,121.78
171181,1729609200,15.96,0.0,0.0,0.0,0.0,290.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,121.78
171182,1729609200,19.90,17.7,17.7,87.0,17.6,293.0,21.6,60.8,0.0,...,False,False,True,False,False,False,False,False,False,121.78


# 데이터 Interpolation (가격)

In [113]:
original_price_list = pd.notna(new_interpolation_data['하루전가격(원/kWh)'])
new_interpolation_data['확정가격여부'] = original_price_list

interpolation_helper = InterpolationHelper(
    new_interpolation_data, 
    ['하루전가격(원/kWh)']
)
new_interpolation_data = interpolation_helper.calculate_interpolation()

new_interpolation_data

100%|████████████████████████████| 171184/171184 [00:49<00:00, 3467.40it/s]


,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Geumak-ri,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri,하루전가격(원/kWh),확정가격여부
0,1709218800,6.90,2.4,2.4,69.0,1.5,338.0,18.7,37.7,0.0,...,False,False,False,True,False,False,False,False,107.39,True
1,1709218800,4.90,-1.8,-1.8,66.0,-1.0,293.0,26.5,31.8,0.0,...,True,False,False,False,False,False,False,False,107.39,True
2,1709218800,7.80,5.7,5.7,65.0,2.2,338.0,11.1,11.1,0.0,...,False,False,False,False,False,False,False,False,107.39,True
3,1709218800,7.10,7.5,7.5,66.0,1.1,0.0,5.0,16.0,0.0,...,False,True,False,False,False,False,False,False,107.39,True
4,1709218800,8.90,2.2,2.2,71.0,3.9,315.0,36.4,36.4,0.0,...,False,False,False,False,False,True,False,False,107.39,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171179,1729609200,27.10,27.4,27.4,70.0,21.3,225.0,17.7,51.7,0.0,...,False,False,False,False,False,False,False,False,121.78,True
171180,1729609200,20.00,16.6,16.6,82.0,17.2,293.0,33.3,55.6,0.0,...,False,False,False,False,False,False,False,False,121.78,True
171181,1729609200,20.20,17.5,17.5,86.0,17.7,293.0,32.4,50.1,0.0,...,False,False,False,True,False,False,False,False,121.78,True
171182,1729609200,20.00,18.5,18.5,84.0,17.4,225.0,12.6,49.7,0.0,...,False,False,True,False,False,False,False,False,121.78,True


# 결측치 제거

In [121]:
data_cleaned = new_interpolation_data.dropna(axis=0)
data_cleaned

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Geumak-ri,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri,하루전가격(원/kWh),확정가격여부
0,1709218800,6.90,2.4,2.4,69.0,1.5,338.0,18.7,37.7,0.0,...,False,False,False,True,False,False,False,False,107.39,True
1,1709218800,4.90,-1.8,-1.8,66.0,-1.0,293.0,26.5,31.8,0.0,...,True,False,False,False,False,False,False,False,107.39,True
2,1709218800,7.80,5.7,5.7,65.0,2.2,338.0,11.1,11.1,0.0,...,False,False,False,False,False,False,False,False,107.39,True
3,1709218800,7.10,7.5,7.5,66.0,1.1,0.0,5.0,16.0,0.0,...,False,True,False,False,False,False,False,False,107.39,True
4,1709218800,8.90,2.2,2.2,71.0,3.9,315.0,36.4,36.4,0.0,...,False,False,False,False,False,True,False,False,107.39,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171179,1729609200,27.10,27.4,27.4,70.0,21.3,225.0,17.7,51.7,0.0,...,False,False,False,False,False,False,False,False,121.78,True
171180,1729609200,20.00,16.6,16.6,82.0,17.2,293.0,33.3,55.6,0.0,...,False,False,False,False,False,False,False,False,121.78,True
171181,1729609200,20.20,17.5,17.5,86.0,17.7,293.0,32.4,50.1,0.0,...,False,False,False,True,False,False,False,False,121.78,True
171182,1729609200,20.00,18.5,18.5,84.0,17.4,225.0,12.6,49.7,0.0,...,False,False,True,False,False,False,False,False,121.78,True


# 데이터 저장하기

In [122]:
new_interpolation_data.to_csv('train_dataset.csv', index=False)
data_cleaned.to_csv('train_dataset_clean.csv', index=False)

# 가격 데이터가 존재하는 컬럼만 추출

In [123]:
only_has_price = data_cleaned[data_cleaned['확정가격여부'] == True]
only_has_price

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Geumak-ri,location_Gwangryeong-ri,location_Hacheon-ri,location_Ilgwa-ri,location_Jeju,location_Sangmo-ri,location_Songdang-ri,location_Yongsu-ri,하루전가격(원/kWh),확정가격여부
0,1709218800,6.90,2.4,2.4,69.0,1.5,338.0,18.7,37.7,0.0,...,False,False,False,True,False,False,False,False,107.39,True
1,1709218800,4.90,-1.8,-1.8,66.0,-1.0,293.0,26.5,31.8,0.0,...,True,False,False,False,False,False,False,False,107.39,True
2,1709218800,7.80,5.7,5.7,65.0,2.2,338.0,11.1,11.1,0.0,...,False,False,False,False,False,False,False,False,107.39,True
3,1709218800,7.10,7.5,7.5,66.0,1.1,0.0,5.0,16.0,0.0,...,False,True,False,False,False,False,False,False,107.39,True
4,1709218800,8.90,2.2,2.2,71.0,3.9,315.0,36.4,36.4,0.0,...,False,False,False,False,False,True,False,False,107.39,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171179,1729609200,27.10,27.4,27.4,70.0,21.3,225.0,17.7,51.7,0.0,...,False,False,False,False,False,False,False,False,121.78,True
171180,1729609200,20.00,16.6,16.6,82.0,17.2,293.0,33.3,55.6,0.0,...,False,False,False,False,False,False,False,False,121.78,True
171181,1729609200,20.20,17.5,17.5,86.0,17.7,293.0,32.4,50.1,0.0,...,False,False,False,True,False,False,False,False,121.78,True
171182,1729609200,20.00,18.5,18.5,84.0,17.4,225.0,12.6,49.7,0.0,...,False,False,True,False,False,False,False,False,121.78,True


In [124]:
only_has_price.to_csv('train_dataset_price_only.csv', index=False)

# 현황데이터도 merge

In [125]:
conditions = pd.read_csv('../OIBC_2024_DATA/data/제주전력시장_현황데이터.csv')
conditions

,ts,공급능력(kW),현재 수요(kW),태양광 발전량kW),풍력 발전량(kW),신재생 발전량 총합(kW),공급 예비력(kW),운영 예비력(kW)
0,1704034800,1298000.0,769000.0,0.0,125426.0,133103.0,529000.0,323000.0
1,1704035100,1299000.0,761000.0,0.0,130393.0,137867.0,538000.0,339000.0
2,1704035400,1290000.0,755000.0,0.0,120691.0,128133.0,534000.0,335000.0
3,1704035700,1277000.0,756000.0,0.0,107771.0,115519.0,521000.0,325000.0
4,1704036000,1269000.0,753000.0,0.0,99264.4,107153.0,515000.0,318000.0
...,...,...,...,...,...,...,...,...
84075,1729607700,1254000.0,624000.0,0.0,247628.0,256210.0,625000.0,277000.0
84076,1729608000,1248000.0,618000.0,0.0,246598.0,255217.0,627000.0,277000.0
84077,1729608300,1255000.0,621000.0,0.0,251494.0,260161.0,630000.0,281000.0
84078,1729608600,1259000.0,612000.0,0.0,252912.0,261535.0,642000.0,290000.0


In [131]:
data_with_conditions = pd.merge(only_has_price, conditions, on='ts', how='inner')
data_with_conditions

,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,...,location_Yongsu-ri,하루전가격(원/kWh),확정가격여부,공급능력(kW),현재 수요(kW),태양광 발전량kW),풍력 발전량(kW),신재생 발전량 총합(kW),공급 예비력(kW),운영 예비력(kW)
0,1709218800,6.90,2.4,2.4,69.0,1.5,338.0,18.7,37.7,0.0,...,False,107.39,True,1388000.0,798000.0,0.0,234165.0,248660.0,590000.0,266000.0
1,1709218800,4.90,-1.8,-1.8,66.0,-1.0,293.0,26.5,31.8,0.0,...,False,107.39,True,1388000.0,798000.0,0.0,234165.0,248660.0,590000.0,266000.0
2,1709218800,7.80,5.7,5.7,65.0,2.2,338.0,11.1,11.1,0.0,...,False,107.39,True,1388000.0,798000.0,0.0,234165.0,248660.0,590000.0,266000.0
3,1709218800,7.10,7.5,7.5,66.0,1.1,0.0,5.0,16.0,0.0,...,False,107.39,True,1388000.0,798000.0,0.0,234165.0,248660.0,590000.0,266000.0
4,1709218800,8.90,2.2,2.2,71.0,3.9,315.0,36.4,36.4,0.0,...,False,107.39,True,1388000.0,798000.0,0.0,234165.0,248660.0,590000.0,266000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72067,1729609200,27.10,27.4,27.4,70.0,21.3,225.0,17.7,51.7,0.0,...,False,121.78,True,1254000.0,599000.0,0.0,248589.0,257082.0,650000.0,303000.0
72068,1729609200,20.00,16.6,16.6,82.0,17.2,293.0,33.3,55.6,0.0,...,False,121.78,True,1254000.0,599000.0,0.0,248589.0,257082.0,650000.0,303000.0
72069,1729609200,20.20,17.5,17.5,86.0,17.7,293.0,32.4,50.1,0.0,...,False,121.78,True,1254000.0,599000.0,0.0,248589.0,257082.0,650000.0,303000.0
72070,1729609200,20.00,18.5,18.5,84.0,17.4,225.0,12.6,49.7,0.0,...,False,121.78,True,1254000.0,599000.0,0.0,248589.0,257082.0,650000.0,303000.0


In [132]:
data_with_conditions.to_csv('train_dataset_with_condition.csv', index=False)

# 하루전 가격 + 실시간 가격

In [8]:
하루전가격 = pd.read_csv('../OIBC_2024_DATA/data/제주전력시장_시장전기가격_하루전가격.csv')
실시간가격 = pd.read_csv('../OIBC_2024_DATA/data/제주전력시장_시장전기가격_실시간가격.csv')

실시간가격 = 실시간가격.drop(columns=['실시간 임시 가격(원/kWh)'])
하루전_실시간 = pd.merge(하루전가격, 실시간가격, on='ts', how='inner')

하루전_실시간.to_csv('days.csv', index=False)